In [1]:
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()

In [2]:
# Triple quotation marks allow code to continue after line break
c.execute("""CREATE TABLE food_table (id INTEGER PRIMARY KEY AUTOINCREMENT, 
          name TEXT, cost INTEGER, calories INTEGER, weight INTEGER)""")

In [3]:
# Insert rows in an alternative way, allows some values to be left unfilled
c.execute("""INSERT INTO food_table(name, cost, calories, weight) VALUES('pizza', 3, 500, 3)""")
c.execute("""INSERT INTO food_table(name, cost, calories) VALUES('burger', 8, 800)""")
c.execute("""INSERT INTO food_table(name, cost, calories, weight) VALUES('fries', 4, 600, 1)""")
c.execute("""INSERT INTO food_table(name, cost, calories, weight) VALUES('fries', 5, 700, 1)""")
c.execute("""INSERT INTO food_table(name, cost, calories, weight) VALUES('broccoli', 1, 20, 4)""")
c.execute("""INSERT INTO food_table(name, cost, calories, weight) VALUES('broccoli', 3, 20, 4)""")


c.execute("SELECT * FROM food_table")
c.fetchall()

[(1, u'pizza', 3, 500, 3),
 (2, u'burger', 8, 800, None),
 (3, u'fries', 4, 600, 1),
 (4, u'fries', 5, 700, 1),
 (5, u'broccoli', 1, 20, 4),
 (6, u'broccoli', 3, 20, 4)]

In [4]:
# Select ... WHERE a AND b
c.execute("SELECT * FROM food_table WHERE calories > 300 AND cost < 5")
c.fetchall()

[(1, u'pizza', 3, 500, 3), (3, u'fries', 4, 600, 1)]

In [5]:
c.execute("SELECT * FROM food_table WHERE calories > 700 OR weight = 1")
c.fetchall()

[(2, u'burger', 8, 800, None),
 (3, u'fries', 4, 600, 1),
 (4, u'fries', 5, 700, 1)]

In [6]:
# IN function, synonymous with 'containing'
c.execute("SELECT * FROM food_table WHERE name IN('pizza','burger')")
c.execute("SELECT * FROM food_table WHERE name NOT IN('pizza','burger')")
c.fetchall()

[(3, u'fries', 4, 600, 1),
 (4, u'fries', 5, 700, 1),
 (5, u'broccoli', 1, 20, 4),
 (6, u'broccoli', 3, 20, 4)]

In [7]:
# SELECTING based on values in a second table
c.execute("CREATE TABLE taste_table (name TEXT, taste TEXT)")
c.execute("INSERT INTO taste_table(name, taste) VALUES('pizza','great')")
c.execute("INSERT INTO taste_table(name, taste) VALUES('burger','great')")
c.execute("INSERT INTO taste_table(name, taste) VALUES('fries','great')")
c.execute("INSERT INTO taste_table(name, taste) VALUES('broccoli','bad')")
c.execute("SELECT * FROM taste_table")
c.fetchall()

[(u'pizza', u'great'),
 (u'burger', u'great'),
 (u'fries', u'great'),
 (u'broccoli', u'bad')]

In [8]:
# SELECT food_table items based on whether they are great in the taste_table (called subquerying)
# using IN (synonymous with containing) as a nested function to first return the names of the foods
c.execute("SELECT * FROM food_table WHERE name IN(SELECT name FROM taste_table WHERE taste = 'great')")
c.fetchall()

[(1, u'pizza', 3, 500, 3),
 (2, u'burger', 8, 800, None),
 (3, u'fries', 4, 600, 1),
 (4, u'fries', 5, 700, 1)]

In [9]:
# LIKE operator and % acts like a wildcard
c.execute("SELECT * FROM food_table WHERE name LIKE('pizz%')")
c.fetchall()

[(1, u'pizza', 3, 500, 3)]

In [11]:
# AS renames aggregate functions, AS can also be used to create new columns (not shown here)
c.execute("SELECT name, SUM(calories) AS Total_calories FROM food_table GROUP BY name")
c.fetchall()

[(u'broccoli', 40), (u'burger', 800), (u'fries', 1300), (u'pizza', 500)]

In [12]:
# HAVING applies a 'WHERE-like' condition on the grouped aggregate
c.execute("SELECT name, SUM(calories) AS Total_calories FROM food_table GROUP BY name HAVING Total_calories > 1000")
c.fetchall()

[(u'fries', 1300)]

In [14]:
# Count aggregate function
c.execute("SELECT name FROM food_TABLE GROUP BY name HAVING COUNT(*)>=2")
c.fetchall()

[(u'broccoli',), (u'fries',)]

In [20]:
# CASE function is like 'If'. Can use it to build new columns
c.execute("""SELECT
          name, taste,
              CASE
                  WHEN taste = 'great' THEN 'eat'
                  WHEN taste = 'bad' THEN 'do not eat'
              END AS 'decision'
          FROM taste_table""")
c.fetchall()

[(u'pizza', u'great', u'eat'),
 (u'burger', u'great', u'eat'),
 (u'fries', u'great', u'eat'),
 (u'broccoli', u'bad', u'do not eat')]